In [1]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
env=AngryBird()

# 720 coeffs

In [2]:
coeffs_temp = np.load("l.npy")

In [3]:
coeffs = np.ones((10, 12, 6))
for i in range(10):
    for j in range(12):
        coeffs[i][j] = coeffs_temp[i][j//3]

In [4]:
seconds = np.arange(11)
pos = np.zeros((11, 2))
pos[:, 0] = seconds * 10
pos[:, 1] = seconds * 20 - 2 * seconds**2
quantiles = np.load("quantiles.npy")
mean_states = np.load("mean_states.npy")

In [5]:
sqr2 = 1/np.sqrt(2)
rot = np.array([[sqr2, -sqr2],
               [sqr2, sqr2]])

In [6]:
mean_states = np.load("mean_states_720.npy")

In [7]:
quantiles = np.load("quantiles_720.npy")
print(quantiles[:, 5])
print(np.searchsorted(quantiles[:, 5],2, side="left"))

[-2.53166791 -1.76423279 -1.22126597 -0.78292571 -0.38405904 -0.00407853
  0.38596131  0.76425896  1.21826352  1.71814687  2.4793811 ]
10


In [8]:
def main(state, coeffs):
    t = int(state[0])
    a, _ = rot.dot(state[1:] - mean_states[t])
    i = np.searchsorted(quantiles[:, t],a, side="left")
    return coeffs[t, i, :2]*pos[t]-coeffs[t, i, 2:4]*state[1:] + coeffs[t, i, 4:]

In [ ]:
from time import time
nul=np.zeros(2)
m = 100
n = 1000
vents = np.load("vents.npy")
ecart = 0.0001
cos_min = 400.
while cos_min > 100:
    t = -time()
    for i in range(10):
        for l in range(12):
            for k in range(6):
                coeff_change = np.linspace(coeffs[i][l][k]-ecart, coeffs[i][l][k]+ecart, 9)
                costs = np.zeros(9)
                for j in range(9):
                    coeffs[i][l][k] = coeff_change[j]
                    for z in range(m):
                        state = env.reset()
                        env.vent = vents[z]
                        for _ in range(10):
                            force = main(state, coeffs)
                            state, _, _, _ = env.step(force)
                        env.step(nul)
                        costs[j] += env.cost.sum()
                coeffs[i][l][k] = coeff_change[np.argmin(costs)]
    cos = 0.
    for z in range(n):
        state = env.reset(False)
        env.vent = vents[z]
        for _ in range(10):
            force = main(state, coeffs)
            state, _, _, _ = env.step(force)
        env.step(nul)
        cos += env.cost.sum()
    cos /= n
    t += time()
    print(m, n, ecart, costs.min()/m, cos, t/60)
    if cos < cos_min:
        cos_min = cos
        np.save("c_720", coeffs)
    else:
        cos_min = cos + 20
        m += 100
        n += 100
        ecart /= 2

100 1000 0.0001 110.15783572575039 116.52970390445921 26.487443463007608
100 1000 0.0001 110.02340056778694 116.50975825646655 26.66112722158432
100 1000 0.0001 109.89638947611078 116.52569096906522 26.27198135058085
200 1100 5e-05 116.15303048695705 117.55221663774213 52.76536260843277
200 1100 5e-05 116.09969253991594 117.52697575272444 53.082674554983775
200 1100 5e-05 116.05771328071211 117.51951933457157 56.15034688711167
200 1100 5e-05 116.02276379143922 117.5093228520132 55.66919087966283
200 1100 5e-05 115.99120452700697 117.50505697217338 54.81696545283
200 1100 5e-05 115.96368173855912 117.49885003083284 55.1381876150767
200 1100 5e-05 115.93509787913797 117.49764906506607 55.18656104405721
200 1100 5e-05 115.91216362733127 117.49451144913895 55.317476499080655
200 1100 5e-05 115.89258189930096 117.49201970825867 59.70830765962601
200 1100 5e-05 115.87319793126693 117.50116268844579 57.633403944969174
300 1200 2.5e-05 112.97683417387812 116.00324024763712 83.59928089777628
30

In [ ]:
vents = np.load("vents.npy")
mean_states = np.load("mean_states.npy")
nul=np.zeros(2)
costs = 0.
m = 1100
for z in tqdm(range(m)):
    state = env.reset(False)
    env.vent = vents[z]
    for y in range(10):
        force = main(state, coeffs)
        state, _, _, _ = env.step(force)
    plt.scatter(state[1], state[2])
    state, _, _, _ = env.step(nul)
    costs += env.cost.sum()
print(costs/m)